In [ ]:
import os

In [ ]:
%pwd

In [ ]:
os.chdir('..//')

In [ ]:
%pwd

Following the project workflows

In [ ]:
# Entity is the return type of the function

from dataclasses import dataclass
from pathlib import Path

# Creating the entity class
@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    local_data_file: Path
    unzip_dir: Path
    bucket_name: str
    file_name: str

In [ ]:
from Indian_Coin_Detection.constants import *
from Indian_Coin_Detection.utils.common import read_yaml, create_directories

In [ ]:
class ConfigurationManager:
    def __init__(self,
                 config_filepath = CONFIG_FILE_PATH,
                 params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir, 
            bucket_name=config.bucket_name,
            file_name=config.file_name,
        )

        return data_ingestion_config

In [ ]:
import os
#import urllib.request as request
import boto3
import zipfile
from Indian_Coin_Detection import logger
from Indian_Coin_Detection.utils.common import  get_size

In [ ]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config
    
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            s3 = boto3.client('s3')
            try:
                s3.download_file(self.config.bucket_name, 
                                 self.config.file_name, 
                                 self.config.local_data_file)
                logger.info(f"ZIP file downloaded successfully in the directory {self.config.local_data_file} ")
            except Exception as e:
                logger.info(f'Error downloading the ZIP file: {e}')
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")  

    
    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [ ]:
# Whole Pipeline
try:
    config = ConfigurationManager() #First initializing the configuration manager
    data_ingestion_config = config.get_data_ingestion_config()
    #Calling the components which takes dataingestjion config as input
    data_ingestion = DataIngestion(config=data_ingestion_config) 
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e